# Персептрон для задачи классификации

In [9]:
import torch
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchmetrics.classification import Accuracy
import torch.nn.init as init

In [2]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = MNIST(root="./data", train=True, download=True, transform=transform)

train_size = int(0.6 * len(train_dataset))
val_size = int(0.2 * len(train_dataset))
test_size = len(train_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 37.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.18MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.61MB/s]


In [11]:
class PerceptronClassifier(nn.Module):
  def __init__(self, input_size):
    super(PerceptronClassifier, self).__init__()
    self.fc1 = nn.Linear(input_size, 128)
    self.bn1 = nn.BatchNorm1d(128)

    self.fc2 = nn.Linear(128, 64)
    self.bn2 = nn.BatchNorm1d(64)

    self.fc3 = nn.Linear(64, 32)
    self.bn3 = nn.BatchNorm1d(32)

    self.fc4 = nn.Linear(32, 10)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout()
    self.apply(init_weights)

  def forward(self, x):
    x = self.relu(self.bn1(self.fc1(x)))
    x = self.relu(self.bn2(self.fc2(x)))
    x = self.relu(self.bn3(self.fc3(x)))
    x = self.fc4(x)
    return x

def init_weights(m):
  if isinstance(m, nn.Linear):
    init.xavier_normal_(m.weight)
    if m.bias is not None:
      init.constant_(m.bias, 0)

In [12]:
sample, _ = train_dataset[0]
input_size = sample.view(-1).shape[0]
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PerceptronClassifier(input_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

patience = 10
best_val_loss = float("inf")
patience_counter = 0
model_path = "best_perceptron_classification.pth"

for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct_train = 0
  total_train = 0

  for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images.view(images.size(0), -1))
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    preds = torch.argmax(outputs, dim=1)
    correct_train += (preds == labels).sum().item()
    total_train += labels.size(0)

  train_loss = running_loss /  len(train_loader)
  train_accuracy = correct_train / total_train

  model.eval()
  val_loss = 0.0
  correct_val = 0
  total_val = 0

  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images.view(images.size(0), -1))
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      preds = torch.argmax(outputs, dim=1)
      correct_val += (preds == labels).sum().item()
      total_val += labels.size(0)

  val_loss /= len(val_loader)
  val_accuracy = correct_val / total_val


  print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    patience_counter = 0
    torch.save(model.state_dict(), model_path)
    print(f"✔ Model improved. Saved to {model_path}")
  else:
    patience_counter += 1
    if patience_counter >= patience:
      print("Ранняя остановка сработала!")
      break

model.load_state_dict(torch.load(model_path))
model.to(device)
print(f"✅ Best model loaded from {model_path}")

model.eval()
test_loss = 0.0
correct_test = 0
total_test = 0

with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images.view(images.size(0), -1))
    loss = criterion(outputs, labels)
    test_loss += loss.item()

    preds = torch.argmax(outputs, dim=1)
    correct_test += (preds == labels).sum().item()
    total_test += labels.size(0)

test_loss /= len(test_loader)
test_accuracy = correct_test / total_test
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/50: 100%|██████████| 563/563 [00:12<00:00, 45.04it/s]


Epoch 1, Train Loss: 0.3923, Val Loss: 0.1552
✔ Model improved. Saved to best_perceptron_classification.pth


Epoch 2/50: 100%|██████████| 563/563 [00:12<00:00, 45.46it/s]


Epoch 2, Train Loss: 0.1381, Val Loss: 0.1076
✔ Model improved. Saved to best_perceptron_classification.pth


Epoch 3/50: 100%|██████████| 563/563 [00:13<00:00, 43.10it/s]


...

Epoch 23, Train Loss: 0.0145, Val Loss: 0.1103


Epoch 24/50: 100%|██████████| 563/563 [00:12<00:00, 46.53it/s]


Epoch 24, Train Loss: 0.0130, Val Loss: 0.1013


Epoch 25/50: 100%|██████████| 563/563 [00:11<00:00, 46.94it/s]


Epoch 25, Train Loss: 0.0123, Val Loss: 0.0976
Ранняя остановка сработала!
✅ Best model loaded from best_perceptron_classification.pth
Test Loss: 0.0887, Test Accuracy: 0.9752


In [8]:
from google.colab import files
files.download("best_perceptron_classification.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>